In [1]:
%matplotlib inline
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pink_utils as pu
import pandas as pd
import astropy.units as u
from tqdm import tqdm
from astropy.coordinates import SkyCoord, search_around_sky
from pink_utils import Annotation

In [2]:
FIRST_PIX = 1.8*u.arcsecond # Pixel size of FIRST survey. Square pixels
FIRST_FWHM = 5*u.arcsecond / FIRST_PIX
FIRST_SIG = FIRST_FWHM / 2.355

In [3]:
df        = pd.read_csv('../FIRST_F1W1_95_5_Sources.csv')
transform = pu.transform('../Small/FIRST_F1W1_95_5_Small_Transform.bin')
ed        = pu.heatmap('../Small/FIRST_F1W1_95_5_Small_Similarity.bin')
som       = pu.som('../Small/FIRST_F1W1_95_5_L1_SOM_Small_5.bin')
images    = pu.image_binary('../FIRST_F1W1_95_5_imgs.bin')

with open('../Small/FIRST_F1W1_95_5_L1_SOM_Small_5_Features-table.pkl','rb') as infile:
    annotations = pickle.load(infile)

In [4]:
print(som.file_head)
print(images.file_head)
print(transform.file_head)
print(ed.file_head)
print(df.shape)
NEURON_SIZE = som.file_head[-1] # Equal dimensions 

(2, 12, 12, 1, 118, 118)
(178859, 2, 167, 167)
(178859, 12, 12, 1)
(178859, 12, 12, 1)
(178859, 29)


In [5]:
plt.close('all')        

ms = ['o','*','>']
c  = ['red', 'blue', 'black']
cmap = 'Greys'

for (count, (index, row)) in tqdm(enumerate(df.iterrows())):

    if count != index:
        print('Mistmatch', index, count)
    
#     # Limit reached lets move on
#     if index > 50000:
#         break
        
    cen_pos = SkyCoord(ra=row['RA']*u.deg, dec=row['DEC']*u.deg)
    
    bmu_pos    = ed.get_bmu(index)
    trans_info = transform.get_neuron_transform(index, bmu_pos)
    
    # Annotate_map_features.py script recorded positions around the 
    # incorrect convention
    key = (bmu_pos[1], bmu_pos[0]) + (0,)
    bmu = annotations[key]
    
    # Transform annotated clicks 
    ir_feature_trans = bmu.transform_clicks(trans_info, channel=1)
    
    dx, dy = ir_feature_trans[0]
    comp_pos = pu.estimate_position(cen_pos, dx, dy, pix_scale=FIRST_PIX)

    df.loc[index, 'host.ra'] = comp_pos.ra.deg
    df.loc[index, 'host.dec'] = comp_pos.dec.deg    

178859it [47:20, 62.97it/s]


In [6]:
mask = np.isfinite(df['host.ra'])

host_positions = SkyCoord(ra=df.loc[mask, 'host.ra'].values*u.deg, 
                          dec=df.loc[mask,'host.dec'].values*u.deg, frame='icrs')
host_results = search_around_sky(host_positions, host_positions, seplimit=5*u.arcsec)

df[mask].to_csv('Test.csv')

In [7]:
host_results[0].shape

(184193,)

In [8]:
mask.shape

(178859,)